In [157]:
import os
import numpy as np
import pandas as pd

os.chdir('C:/Users/daeyu/Desktop/FDI_Uncertainty')

# Load the twindustry_to_isic file. (4-digit)

In [158]:
df_indCode = pd.read_excel('twindustry_to_isic.xlsx', header=3)
# print(df_indCode)

In [159]:
df_indCode = df_indCode.replace(r'^\s*$', np.nan, regex=True)  # Replace empty strings with NaN. (Important!)
df_indCode = df_indCode.loc[df_indCode['細類\nClass'].notna()]
# print(df_indCode)

In [160]:
df_indCode = df_indCode.drop(columns=['大類\nSection', '中類\nDivision', '小類\nGroup', 'Unnamed: 5', '大類\nSection.1', '中類\nDivision.1', '小類\nGroup.1'])
# print(df_indCode)

# The code below doesn't work (don't know why):
# df_indCode.drop(columns=['大類\nSection', '中類\nDivision', '小類\nGroup', 'Unnamed: 5', '大類\nSection.1', '中類\nDivision.1', '小類\nGroup.1'])

In [161]:
df_indCode = df_indCode.rename(columns={'細類\nClass':'twIndCode_4d', 'Unnamed: 4':'twIndName', '細類\nClass.1':'ISIC_4d', 'Unnamed: 10':'partial_ref', 'Unnamed: 11':'ISICIndName', 'Unnamed: 12':'ISICIndDes'})

In [162]:
# Reset the index.
# 這個步驟太重要了，不然後續在比對industry name的時候會跑出「Can only compare identically-labeled Series objects」的錯誤訊息。
df_indCode = df_indCode.reset_index(drop=True)
print(df_indCode)

    twIndCode_4d twIndName  ISIC_4d partial_ref            ISICIndName  \
0           0111     稻作栽培業    112.0         NaN                  稻米栽培業   
1           0112     雜糧栽培業    111.0           *  穀類（稻米除外）、豆科作物及油料種子栽培業   
2           0113   特用作物栽培業    111.0           *  穀類（稻米除外）、豆科作物及油料種子栽培業   
3           0113   特用作物栽培業    114.0         NaN                  甘蔗栽培業   
4           0113   特用作物栽培業    115.0         NaN                  菸草栽培業   
..           ...       ...      ...         ...                    ...   
644         9622     美容美體業   9602.0           *                 理髮及美容業   
645         9622     美容美體業   9609.0           *               未分類其他服務業   
646         9630  殯葬及相關服務業   9603.0         NaN               殯葬及相關服務業   
647         9640     家事服務業   9700.0         NaN                  家事服務業   
648         9690   其他個人服務業   9609.0           *               未分類其他服務業   

                                            ISICIndDes  
0                                      Growing of rice

In [163]:
print(df_indCode['twIndCode_4d'].isnull().sum())
print(df_indCode['ISIC_4d'].isnull().sum())

0
0


In [164]:
for column in df_indCode.columns:
    print(column,':',pd.api.types.infer_dtype(df_indCode[column]))

twIndCode_4d : mixed-integer
twIndName : string
ISIC_4d : floating
partial_ref : string
ISICIndName : string
ISICIndDes : string


In [165]:
df_indCode['twIndCode_4d'] = df_indCode['twIndCode_4d'].astype(str)

df_indCode['ISIC_4d'] = df_indCode['ISIC_4d'].astype(str)
df_indCode['ISIC_4d'] = df_indCode['ISIC_4d'].str.replace(".0", "")
df_indCode['ISIC_4d'] = df_indCode['ISIC_4d'].apply('{:0>4}'.format)

print(df_indCode)

    twIndCode_4d twIndName ISIC_4d partial_ref            ISICIndName  \
0           0111     稻作栽培業    0112         NaN                  稻米栽培業   
1           0112     雜糧栽培業    0111           *  穀類（稻米除外）、豆科作物及油料種子栽培業   
2           0113   特用作物栽培業    0111           *  穀類（稻米除外）、豆科作物及油料種子栽培業   
3           0113   特用作物栽培業    0114         NaN                  甘蔗栽培業   
4           0113   特用作物栽培業    0115         NaN                  菸草栽培業   
..           ...       ...     ...         ...                    ...   
644         9622     美容美體業    0092           *                 理髮及美容業   
645         9622     美容美體業    0099           *               未分類其他服務業   
646         9630  殯葬及相關服務業    0093         NaN               殯葬及相關服務業   
647         9640     家事服務業    0090         NaN                  家事服務業   
648         9690   其他個人服務業    0099           *               未分類其他服務業   

                                            ISICIndDes  
0                                      Growing of rice  
1    Grow

C:\Users\daeyu\AppData\Local\Temp\ipykernel_3828\467142149.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_indCode['ISIC_4d'] = df_indCode['ISIC_4d'].str.replace(".0", "")


In [166]:
for column in df_indCode.columns:
    print(column,':',pd.api.types.infer_dtype(df_indCode[column]))

twIndCode_4d : string
twIndName : string
ISIC_4d : string
partial_ref : string
ISICIndName : string
ISICIndDes : string


# Load the FDI data.

In [167]:
df_FDI = pd.read_excel('上市櫃公司對外和對中國大陸投資事業名錄_precleaned.xlsx', na_values=[""], header=0)

print(df_FDI)

          國別            業別                       對外投資事業名稱中文         國內投資人  \
0         美國       生物藥品製造業                     MEDINOX INC.  理隆纖維工業股份有限公司   
1         美國        原料藥製造業               PHYTOCEUTICA, INC.  理隆纖維工業股份有限公司   
2         美國         西藥製造業  AMPHASTAR PHARMACEUTICALS, INC.  理隆纖維工業股份有限公司   
3         美國       積體電路製造業                          美商聯特利公司  理隆纖維工業股份有限公司   
4     英屬維京群島  電子設備及其零組件批發業         LILON INVESTMENT LIMITED  理隆纖維工業股份有限公司   
...      ...           ...                              ...           ...   
9551  英屬開曼群島    運動用品、器材批發業                          愛地雅開曼公司   愛地雅工業股份有限公司   
9552      中國      自行車零件製造業                   領先運動器材（東莞）有限公司   愛地雅工業股份有限公司   
9553      波蘭    運動用品、器材批發業                        愛地雅波蘭有限公司   愛地雅工業股份有限公司   
9554      德國       機車零件製造業                    INNOTORQ GMBH   愛地雅工業股份有限公司   
9555      英國        自行車製造業          KARBON KINETICS LIMITED   愛地雅工業股份有限公司   

          統一編號     核准日期                                             對外事業地址 

In [168]:
df_FDI = df_FDI.rename(columns={'業別': 'indName_recorded'})

print(df_FDI['indName_recorded'].isnull().sum())

441


In [169]:
for column in df_FDI.columns:
    print(column,':',pd.api.types.infer_dtype(df_FDI[column]))

國別 : string
indName_recorded : string
對外投資事業名稱中文 : string
國內投資人 : string
統一編號 : integer
核准日期 : integer
對外事業地址 : string
國內地址 : string
身份註記 : string
主要營業項目 : string
公司名稱_TEJ : string
主計處產業代碼_TEJ : string
主計處產業名稱_TEJ : string
員工人數母公司2023_TEJ : integer
設立日期_TEJ : string
主要經營業務2023_TEJ : string
公司中文全稱_TEJ : string


# Start matching the industry names in df_FDI and df_indCode. (first for Taiwan-classification only)

In [170]:
from collections import Counter

df_FDI['twIndName_official'] = np.nan

cannotFind_list = []
NaN_count = 0

for i in df_FDI.index:
    value = df_FDI['indName_recorded'].iloc[i]
    if pd.notna(value) == True:
        
        indName_i = df_FDI['indName_recorded'][i]
        print(indName_i, end =" ")
        print(df_indCode.index[(df_indCode['twIndName']==indName_i)], end ="; ")
        
        if len(df_indCode.index[(df_indCode['twIndName']==indName_i)]) >= 1:
            print(df_indCode.index[(df_indCode['twIndName']==indName_i)][0])
            df_FDI['twIndName_official'][i] = df_indCode['twIndName'][df_indCode.index[(df_indCode['twIndName']==indName_i)][0]]
            
        else:  # i.e. if len(df_indCode.index[(df_indCode['twIndName']==indName_i)]) == 0:
            print("Cannot find the industry in the table!!")
            cannotFind_list.append(indName_i)

    else:
        print("No industry name recorded!")
        NaN_count += 1


# Organize the cannotFind_list into a dataframe.
checked_list = []
cannotFind_count = []
for m in cannotFind_list:
    alreadyChecked = 0
    for i in range(len(checked_list)):
        if m == checked_list[i]:
            cannotFind_count[i] += 1
            alreadyChecked = True
            break
    if alreadyChecked == False:
        checked_list.append(m)
        cannotFind_count.append(1)
        
toPrint_df = pd.DataFrame(list(zip(checked_list, cannotFind_count)), columns=['業別', 'count'])
toPrint_df = toPrint_df.sort_values('count', ascending=False)
toPrint_df = toPrint_df.reset_index(drop=True)


# Output
print("---------------------------------------------------------------------------")
print("Number of observartions without industry entry: " + str(NaN_count))
print("---------------------------------------------------------------------------")
print(toPrint_df)

os.chdir('C:/Users/daeyu/Desktop/FDI_Uncertainty/Report')
toPrint_df.to_excel('Industries_with_no_perfect_match.xlsx', index=False)
os.chdir('C:/Users/daeyu/Desktop/FDI_Uncertainty')

# Can check with the "Counter" function.
# print("----------------------------------------------------------")
# print(Counter(cannotFind_list).most_common())

生物藥品製造業 Int64Index([], dtype='int64'); Cannot find the industry in the table!!
原料藥製造業 Int64Index([132], dtype='int64'); 132
西藥製造業 Int64Index([133], dtype='int64'); 133
積體電路製造業 Int64Index([187], dtype='int64'); 187
電子設備及其零組件批發業 Int64Index([], dtype='int64'); Cannot find the industry in the table!!
海洋水運業 Int64Index([416, 417], dtype='int64'); 416
生物藥品製造業 Int64Index([], dtype='int64'); Cannot find the industry in the table!!
金融控股業 Int64Index([480], dtype='int64'); 480
生物藥品製造業 Int64Index([], dtype='int64'); Cannot find the industry in the table!!
化學原料批發業 Int64Index([], dtype='int64'); Cannot find the industry in the table!!
其他金融輔助業 Int64Index([], dtype='int64'); Cannot find the industry in the table!!
電腦製造業 Int64Index([198], dtype='int64'); 198
企業總管理機構 Int64Index([519], dtype='int64'); 519
航空貨運承攬業 Int64Index([426], dtype='int64'); 426
航空貨運承攬業 Int64Index([426], dtype='int64'); 426
航空貨運承攬業 Int64Index([426], dtype='int64'); 426
航空貨運承攬業 Int64Index([426], dtype='int64'); 426
航空貨運承攬業 Int64Index(

C:\Users\daeyu\AppData\Local\Temp\ipykernel_3828\3745334427.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_FDI['twIndName_official'][i] = df_indCode['twIndName'][df_indCode.index[(df_indCode['twIndName']==indName_i)][0]]
C:\Users\daeyu\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Int64Index([437], dtype='int64'); 437
投資顧問業 Int64Index([503, 506], dtype='int64'); 503
金融控股業 Int64Index([480], dtype='int64'); 480
合成樹脂及塑膠製造業 Int64Index([], dtype='int64'); Cannot find the industry in the table!!
未分類其他電子零組件製造業 Int64Index([197], dtype='int64'); 197
其他金融輔助業 Int64Index([], dtype='int64'); Cannot find the industry in the table!!
其他未分類製造業 Int64Index([269], dtype='int64'); 269
銅材軋延、擠型、伸線業 Int64Index([], dtype='int64'); Cannot find the industry in the table!!
電子管製造業 Int64Index([], dtype='int64'); Cannot find the industry in the table!!
文具製造業 Int64Index([263], dtype='int64'); 263
其他醫療器材及用品製造業 Int64Index([265], dtype='int64'); 265
金融控股業 Int64Index([480], dtype='int64'); 480
未分類其他紙製品製造業 Int64Index([116], dtype='int64'); 116
石油及煤製品製造業 Int64Index([120, 121], dtype='int64'); 120
未分類其他紙製品製造業 Int64Index([116], dtype='int64'); 116
化學製品批發業 Int64Index([], dtype='int64'); Cannot find the industry in the table!!
其他化學製品製造業 Int64Index([], dtype='int64'); Cannot find the industry in the tabl

In [171]:
print("Number of observations with industry entry that cannot be identified:  " + str(sum(cannotFind_count)))

Number of observations with industry entry that cannot be identified:  2441


In [172]:
print("Number of observations whose industry cannot be identified:  " + str(df_FDI['twIndName_official'].isnull().sum()))


Number of observations whose industry cannot be identified:  2882


In [173]:
for column in df_FDI.columns:
    print(column,':',pd.api.types.infer_dtype(df_FDI[column]))

國別 : string
indName_recorded : string
對外投資事業名稱中文 : string
國內投資人 : string
統一編號 : integer
核准日期 : integer
對外事業地址 : string
國內地址 : string
身份註記 : string
主要營業項目 : string
公司名稱_TEJ : string
主計處產業代碼_TEJ : string
主計處產業名稱_TEJ : string
員工人數母公司2023_TEJ : integer
設立日期_TEJ : string
主要經營業務2023_TEJ : string
公司中文全稱_TEJ : string
twIndName_official : string


# First-two-last-three (F2L3) Method

In [174]:
def strcheck_firstTwolastThree(str1, str2):
    if (str1[0:2]==str2[0:2] and str1[-3:]==str2[-3:]):
        return True
    else:
        return False

In [175]:
unidentifiedAfterF2L3_list = []
identifiedAfterF2L3_list = []

for i in df_FDI.index:
    if pd.isna(df_FDI['twIndName_official'][i]) == True:
        # print(df_FDI['業別'][i], end=';')
        
        if pd.isna(df_FDI['indName_recorded'][i]) == False:
            print( df_FDI['indName_recorded'][i] + " has not been identified." )
            industry_candidate_list = []
            industry_candidateIndex_list = []
            
            if(df_FDI['indName_recorded'][i][0:2] != "其他"):
                for j in df_indCode.index:
                    if strcheck_firstTwolastThree(df_FDI['indName_recorded'][i], df_indCode['twIndName'][j]) == True:
                        industry_candidate_list.append(df_indCode['twIndName'][j])
                        industry_candidateIndex_list.append(j)
                        print( df_FDI['indName_recorded'][i] + " can be identified as " + df_indCode['twIndName'][j] )
                #print("--------------------")
            else:  #i.e. if(df_FDI['業別'][i][0:2] == "其他"):
                for j in df_indCode.index:
                    if df_indCode['twIndName'][j][0:2] == "其他":
                        if strcheck_firstTwolastThree(df_FDI['indName_recorded'][i][2:], df_indCode['twIndName'][j][2:]) == True:
                            industry_candidate_list.append(df_indCode['twIndName'][j])
                            industry_candidateIndex_list.append(j)
                #print("--------------------")

            if len(set(industry_candidate_list)) == 1:
                print(df_FDI['indName_recorded'][i] + " is surely identified as " + df_indCode['twIndName'][industry_candidateIndex_list[0]])
                df_FDI['twIndName_official'][i] = df_indCode['twIndName'][industry_candidateIndex_list[0]]
                identifiedAfterF2L3_list.append([df_FDI['indName_recorded'][i], industry_candidate_list[0]])
            else:  #i.e. if len(set(industry_candidate_list))==0 or len(set(industry_candidate_list))>1:
                unidentifiedAfterF2L3_list.append(df_FDI['indName_recorded'][i])
                
            print("---------------------------------------------------------")
        else:
            continue
    
    else:
        continue

生物藥品製造業 has not been identified.
---------------------------------------------------------
電子設備及其零組件批發業 has not been identified.
電子設備及其零組件批發業 can be identified as 電子、通訊設備及其零組件批發業
電子設備及其零組件批發業 is surely identified as 電子、通訊設備及其零組件批發業
---------------------------------------------------------
生物藥品製造業 has not been identified.
---------------------------------------------------------
生物藥品製造業 has not been identified.
---------------------------------------------------------
化學原料批發業 has not been identified.
化學原料批發業 can be identified as 化學原材料及其製品批發業
化學原料批發業 is surely identified as 化學原材料及其製品批發業
---------------------------------------------------------
其他金融輔助業 has not been identified.
---------------------------------------------------------
塑膠皮、板、管材製造業 has not been identified.
塑膠皮、板、管材製造業 can be identified as 塑膠原料製造業
塑膠皮、板、管材製造業 can be identified as 塑膠皮、板及管材製造業
塑膠皮、板、管材製造業 can be identified as 塑膠膜袋製造業
塑膠皮、板、管材製造業 can be identified as 塑膠外殼及配件製造業
---------------------------------------------------

In [176]:
# Report the result.

from collections import Counter

checked_before = []
col1_recorded = []
col2_official = []
col3_count = []
for m in identifiedAfterF2L3_list:
    alreadyChecked = 0
    for i in range(len(checked_before)):
        if m[0] == checked_before[i][0]:
            col3_count[i] += 1
            alreadyChecked = True
            break
    if alreadyChecked == False:
        checked_before.append(m)
        col1_recorded.append(m[0])
        col2_official.append(m[1])
        col3_count.append(1)

# print(identifiedAfterF2L3_count)
toPrint_df = {'對外投資事業名錄': col1_recorded, 'twindustry_to_isic': col2_official, 'count': col3_count}
toPrint_df = pd.DataFrame(toPrint_df)
toPrint_df = toPrint_df.sort_values('count', ascending=False)
toPrint_df = toPrint_df.reset_index(drop=True)
print(toPrint_df)

os.chdir('C:/Users/daeyu/Desktop/FDI_Uncertainty/Report')
toPrint_df.to_excel('IdentifiedAfterF2L3.xlsx', index=False)
os.chdir('C:/Users/daeyu/Desktop/FDI_Uncertainty')


print("************************************************************************************************************")
print("************************************************************************************************************")
print("************************************************************************************************************")

# Report the industries that cannot be identified using F2L3.
checked_list = []
unidentifiedAfterF2L3_count = []
for m in unidentifiedAfterF2L3_list:
    alreadyChecked = 0
    for i in range(len(checked_list)):
        if m == checked_list[i]:
            unidentifiedAfterF2L3_count[i] += 1
            alreadyChecked = True
            break
    if alreadyChecked == False:
        checked_list.append(m)
        unidentifiedAfterF2L3_count.append(1)
        
toPrint_df = pd.DataFrame(list(zip(checked_list, unidentifiedAfterF2L3_count)), columns=['業別', 'count'])
toPrint_df = toPrint_df.sort_values('count', ascending=False)
toPrint_df = toPrint_df.reset_index(drop=True)
print(toPrint_df)

os.chdir('C:/Users/daeyu/Desktop/FDI_Uncertainty/Report')
toPrint_df.to_excel('UnidentifiedAfterF2L3.xlsx', index=False)
os.chdir('C:/Users/daeyu/Desktop/FDI_Uncertainty')

# Can check with the "Counter" function.
# unidentifiedAfterF2L3_list_counter = Counter(unidentifiedAfterF2L3_list)
# print(unidentifiedAfterF2L3_list_counter.most_common())

           對外投資事業名錄 twindustry_to_isic  count
0      電子設備及其零組件批發業    電子、通訊設備及其零組件批發業    291
1     發電、輸電、配電機械製造業      發電、輸電及配電機械製造業    180
2           電腦軟體設計業            電腦程式設計業     95
3        合成樹脂及塑膠製造業          合成橡膠原料製造業     79
4       其他電腦系統設計服務業          其他電腦相關服務業     61
5            電子管製造業   電子及半導體生產用機械設備製造業     56
6         其他電力設備製造業       其他電力設備及配備製造業     47
7       銅材軋延、擠型、伸線業        銅材軋延、擠型及伸線業     44
8      汽機車零配件、用品批發業       汽機車零配件及用品批發業     37
9        工業用塑膠製品製造業         工業用橡膠製品製造業     36
10       運動用品、器材批發業         運動用品及器材批發業     29
11  資料處理、網站代管及相關服務業    資料處理、主機及網站代管服務業     26
12      鋁材軋延、擠型、伸線業        鋁材軋延、擠型及伸線業     25
13          化學原料批發業       化學原材料及其製品批發業     24
14          家用電扇製造業          家用空調器具製造業     17
15          化學製品批發業       化學原材料及其製品批發業     14
16        石油化工原料製造業          石油及煤製品製造業     13
17  其他食品及飲料、菸草製品零售業    其他食品、飲料及菸草製品零售業     12
18      未分類其他無店面零售業        未分類其他非店面零售業     12
19       運動用品、器材零售業         運動用品及器材零售業     11
20        船舶及其零件製造業         船舶及浮動設

In [177]:
print("Number of observations (with industry entry) that are newly identified after F2L3:  " + str(sum(col3_count)))
print("Number of observations (with industry entry) that cannot be identified even after F2L3:  " + str(sum(unidentifiedAfterF2L3_count)))

print("\n")
print("Number of observations (with and without industry entry) that cannot be identified after F2L3:  " + str(sum(unidentifiedAfterF2L3_count)+df_FDI['indName_recorded'].isnull().sum()))

Number of observations (with industry entry) that are newly identified after F2L3:  1207
Number of observations (with industry entry) that cannot be identified even after F2L3:  1234


Number of observations (with and without industry entry) that cannot be identified after F2L3:  1675


# Levenshtein Distance

In [178]:
def list_to_ordered_set(lst):
    seen = set()
    ordered_set = []
    for item in lst:
        if item not in seen:
            ordered_set.append(item)
            seen.add(item)
    return ordered_set

In [179]:
# Use Levenshtein Distance to identify industries that are not identified even after F2L3. 
import textdistance as td

alreadyChecked_list = []

for i in df_FDI.index:
    if pd.isna(df_FDI['twIndName_official'][i]) == True:
        
        if pd.isna(df_FDI['indName_recorded'][i]) == False:
            
            alreadyChecked = False
            for k in range(len(alreadyChecked_list)):
                if df_FDI['indName_recorded'][i] == alreadyChecked_list[k][0]:
                    alreadyChecked = True
                    indices_we_want_in_df_indCode = df_indCode.index[df_indCode['twIndName']==alreadyChecked_list[k][2][0]] #先隨便assign，之後會再肉眼檢查
                    df_FDI['twIndName_official'][i] = df_indCode['twIndName'][indices_we_want_in_df_indCode[0]]
                    print(df_FDI['indName_recorded'][i] + " has already been identified as: " + alreadyChecked_list[k][0])
                    break
            if alreadyChecked == True:
                continue
                
            # print( df_FDI['業別'][i] + " still has not been identified." )
            industry_candidate_list = []
            industry_candidateIndex_list = []
            min_LvDist = 100
            
            for j in df_indCode.index:
                LvDist = td.levenshtein(df_FDI['indName_recorded'][i], df_indCode['twIndName'][j])
                if LvDist < min_LvDist:
                    industry_candidate_list.clear()
                    industry_candidate_list.append(df_indCode['twIndName'][j])
                    industry_candidateIndex_list.clear()
                    industry_candidateIndex_list.append(j)
                    min_LvDist = LvDist
                elif LvDist == min_LvDist:
                    industry_candidate_list.append(df_indCode['twIndName'][j])
                    industry_candidateIndex_list.append(j)
                else:
                    continue
                    
            print(df_FDI['indName_recorded'][i]+" || ", end="")
            print(list_to_ordered_set(industry_candidate_list), end="")
            print(" || LvDist = "+str(min_LvDist))
            # print("----------------------------------")
            
            indices_we_want_in_df_indCode = df_indCode.index[df_indCode['twIndName']==industry_candidate_list[0]]
            df_FDI['twIndName_official'][i] = df_indCode['twIndName'][indices_we_want_in_df_indCode[0]]
            print(df_FDI['indName_recorded'][i] + " has been identified as: " + df_FDI['twIndName_official'][i])
           
            if len(set(industry_candidate_list)) == 1:
                alreadyChecked_list.append([df_FDI['indName_recorded'][i], str(min_LvDist), list_to_ordered_set(industry_candidate_list), ""])  
            else:
                alreadyChecked_list.append([df_FDI['indName_recorded'][i], str(min_LvDist), list_to_ordered_set(industry_candidate_list), "Y"])
                    
        else:
            continue
    
    else:
        continue

生物藥品製造業 || ['動物飼品製造業'] || LvDist = 2
生物藥品製造業 has been identified as: 動物飼品製造業
生物藥品製造業 has already been identified as: 生物藥品製造業
生物藥品製造業 has already been identified as: 生物藥品製造業
其他金融輔助業 || ['其他運輸輔助業', '其他保險輔助業'] || LvDist = 2
其他金融輔助業 has been identified as: 其他運輸輔助業
塑膠皮、板、管材製造業 || ['塑膠皮、板及管材製造業'] || LvDist = 1
塑膠皮、板、管材製造業 has been identified as: 塑膠皮、板及管材製造業
生物藥品製造業 has already been identified as: 生物藥品製造業
其他金融輔助業 has already been identified as: 其他金融輔助業
信託服務業 || ['會計服務業', '建築服務業', '綠化服務業', '家事服務業'] || LvDist = 2
信託服務業 has been identified as: 會計服務業
其他證券業 || ['其他畜牧業', '其他紡紗業', '其他織布業', '其他住宿業', '其他出版業', '其他電信業', '其他控股業'] || LvDist = 2
其他證券業 has been identified as: 其他畜牧業
其他金融輔助業 has already been identified as: 其他金融輔助業
家庭電器批發業 || ['家用電器批發業'] || LvDist = 1
家庭電器批發業 has been identified as: 家用電器批發業
針織外衣製造業 || ['成衣製造業'] || LvDist = 3
針織外衣製造業 has been identified as: 成衣製造業
水泥製品製造業 || ['磨粉製品製造業', '水泥製造業', '石材製品製造業'] || LvDist = 2
水泥製品製造業 has been identified as: 磨粉製品製造業
其他金融輔助業 has already been identified 

In [180]:
toPrint_df = pd.DataFrame(alreadyChecked_list, columns=['對外投資事業名錄', 'LV distance', 'twindustry_to_isic', 'needs_manual_revision'])
toPrint_df = toPrint_df.sort_values('LV distance')
# with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.precision', 3,):
#     print(toPrint_df)

os.chdir('C:/Users/daeyu/Desktop/FDI_Uncertainty/Report')
toPrint_df.to_excel('ResultAfterLVdist.xlsx', index=False)
os.chdir('C:/Users/daeyu/Desktop/FDI_Uncertainty')

print(toPrint_df)

           對外投資事業名錄 LV distance  \
34              餐館業           1   
46  紡織品、服裝及鞋類之零售攤販業           1   
24          家庭電器零售業           1   
69         無害廢棄物清除業           1   
35          石膏製品製造業           1   
..              ...         ...   
57    其他業務及辦公室支援服務業           6   
21        電腦系統整合服務業           6   
48        砂、石及黏土採取業           6   
23       其他礦業及土石採取業           6   
20   食品飲料為主之綜合商品零售業           7   

                                   twindustry_to_isic needs_manual_revision  
34                                               [餐館]                        
46                                   [紡織品、服裝及鞋類之零售攤販]                        
24                                          [家用電器零售業]                        
69                                         [有害廢棄物清除業]                        
35                                          [石材製品製造業]                        
..                                                ...                   ...  
57                               

In [181]:
print("Number of observations with no industry entry:  " + str(df_FDI['twIndName_official'].isnull().sum()))
print("Number of observations whose industry has not been identified:  " + str(df_FDI['indName_recorded'].isnull().sum()))

# The two numbers above are same. Thus, we have already assigned ISIC_4d to all of the observations with industry entry.

Number of observations with no industry entry:  441
Number of observations whose industry has not been identified:  441


# Check manually.

In [190]:
df_temp = df_FDI.copy(deep=True)

Revise the result from F2L3 method.

In [191]:
F2L3_reviseList_originalValue = ["合成樹脂及塑膠製造業", "其他電腦系統設計服務業",
                                 "電子管製造業", "工業用塑膠製品製造業",
                                 "家用電扇製造業", "工業及研磨材料製造業",
                                 "橡膠製品製造業"]


F2L3_reviseList_newValue = ["塑膠原料製造業", "電腦諮詢及設備管理業",
                           "未分類其他電子零組件製造業", "塑膠皮、板及管材製造業",
                           "其他家用電器製造業", "研磨材料製造業",
                           "橡膠製品製造業"]


for i in range(len(F2L3_reviseList_originalValue)):
    df_check = df_temp.copy(deep=True)
    df_temp.loc[df_temp['indName_recorded'] == F2L3_reviseList_originalValue[i], 'twIndName_official'] = F2L3_reviseList_newValue[i]
    print(F2L3_reviseList_originalValue[i] + " is identified as " + F2L3_reviseList_newValue[i] + " after manual check.")
    count_changes = (df_temp['twIndName_official'] != df_check['twIndName_official']).sum() - 441
    print(f"Number of real changes made: {count_changes}")
    print("------------------------------------------------------------------------------------")


合成樹脂及塑膠製造業 is identified as 塑膠原料製造業 after manual check.
Number of real changes made: 79
------------------------------------------------------------------------------------
其他電腦系統設計服務業 is identified as 電腦諮詢及設備管理業 after manual check.
Number of real changes made: 61
------------------------------------------------------------------------------------
電子管製造業 is identified as 未分類其他電子零組件製造業 after manual check.
Number of real changes made: 56
------------------------------------------------------------------------------------
工業用塑膠製品製造業 is identified as 塑膠皮、板及管材製造業 after manual check.
Number of real changes made: 36
------------------------------------------------------------------------------------
家用電扇製造業 is identified as 其他家用電器製造業 after manual check.
Number of real changes made: 17
------------------------------------------------------------------------------------
工業及研磨材料製造業 is identified as 研磨材料製造業 after manual check.
Number of real changes made: 2
---------------------------------------

Revise the result from Levenshtein distance.

In [192]:
LVDist_reviseList_originalValue = [
                           "無害廢棄物清除業","石膏製品製造業", "藥品製造業", "旅行業",
                           "證券金融業", "肉品製造業", "其他服飾品製造業", "化粧品製造業",
                           "非酒精飲料店業", "其他餐飲業", "生物藥品製造業", "未分類其他金融中介業",
                           "保全服務業", "未分類其他教育服務業", "清潔服務業", "普通航空業",
                           "紙容器製造業", "其他金融輔助業", "信託服務業", "其他證券業", "水產品製造業",
                           "水泥製品製造業", "蔬果製品製造業", "石灰製造業", "印染整理業", "船舶租賃業", "貨櫃租賃業",
                           "電信業", "金屬彈簧製造業", "其他就業服務業", "其他住宿服務業", "其他化學製品製造業",
                           "金屬手工具製造業", "紡織帽製造業", "護理照顧服務業", "老人居住照顧服務業",
                           "金屬線製品製造業", "其他物品租賃業", "動物飼料配製業", "塑膠日用品製造業", "農產品整理業",
                           "理髮及美容業", "系統保全服務業", "未分類其他資訊供應服務業", "運動及娛樂用品租賃業",
                           "冷凍冷藏肉類製造業", "陶瓷衛浴設備製造業", "冷凍冷藏蔬果製造業", "影片後製服務業", 
                           "石油製品燃料批發業", "其他藝術表演輔助服務業", "其他不能歸類之行業", "體外檢驗試劑製造業",
                           "非金融性無形資產租賃業", "電腦系統整合服務業", "砂、石及黏土採取業", "其他礦業及土石採取業",
                           "食品飲料為主之綜合商品零售業" ]


LVDist_reviseList_newValue = [
                           "非有害廢棄物清除業", "水泥及混凝土製品製造業", "藥品及醫用化學製品製造業", "旅行及相關服務業",
                           "證券商", "屠宰業", "服飾品製造業", "清潔用品及化粧品製造業",
                           "飲料店", "外燴及團膳承包業", "醫用生物製品製造業", "未分類其他金融服務業",
                           "保全及偵探業", "未分類其他教育業", "清潔服務業", "航空運輸業",
                           "瓦楞紙板及紙容器製造業", "其他金融輔助業", "信託、基金及類似金融實體", "證券商", "水產加工及保藏業",
                           "水泥及混凝土製品製造業", "蔬果加工及保藏業", "水泥製造業", "染整業", "其他運輸工具租賃業",
                           "其他機械設備租賃業",
                           "電信業", "金屬彈簧及線製品製造業", "人力仲介業", "其他住宿業", "其他化學製品製造業",
                           "金屬刀具及手工具製造業", "服飾品製造業", "居住型護理照顧服務業", "居住型老人照顧服務業",
                           "金屬彈簧及線製品製造業", "其他機械設備租賃業", "動物飼品製造業", "其他塑膠製品製造業", "農事及畜牧服務業",
                           "美髮業", "保全及偵探業", "其他資訊服務業", "個人及家庭用品租賃業",
                           "肉類其他加工及保藏業", "黏土建築材料製造業", "蔬果加工及保藏業", "影片及電視節目後製業", 
                           "液體、氣體燃料及相關產品批發業", "創作及藝術表演輔助業", "其他個人服務業", "醫用化學製品製造業",
                           "智慧財產租賃業", "電腦諮詢及設備管理業", "砂、石採取及其他礦業", "砂、石採取及其他礦業",
                           "其他綜合商品零售業" ]


for i in range(len(LVDist_reviseList_originalValue)):
    df_check = df_temp.copy(deep=True)
    df_temp.loc[df_temp['indName_recorded'] == LVDist_reviseList_originalValue[i], 'twIndName_official'] = LVDist_reviseList_newValue[i]
    print(LVDist_reviseList_originalValue[i] + " is identified as " + LVDist_reviseList_newValue[i] + " after manual check.")
    count_changes = (df_temp['twIndName_official'] != df_check['twIndName_official']).sum() - 441
    print(f"Number of real changes made: {count_changes}")
    print("------------------------------------------------------------------------------------")

無害廢棄物清除業 is identified as 非有害廢棄物清除業 after manual check.
Number of real changes made: 1
------------------------------------------------------------------------------------
石膏製品製造業 is identified as 水泥及混凝土製品製造業 after manual check.
Number of real changes made: 1
------------------------------------------------------------------------------------
藥品製造業 is identified as 藥品及醫用化學製品製造業 after manual check.
Number of real changes made: 10
------------------------------------------------------------------------------------
旅行業 is identified as 旅行及相關服務業 after manual check.
Number of real changes made: 7
------------------------------------------------------------------------------------
證券金融業 is identified as 證券商 after manual check.
Number of real changes made: 2
------------------------------------------------------------------------------------
肉品製造業 is identified as 屠宰業 after manual check.
Number of real changes made: 2
---------------------------------------------------------------------------

In [193]:
print("Number of observations with no industry entry:  " + str(df_temp['twIndName_official'].isnull().sum()))
print("Number of observations whose industry has not been identified:  " + str(df_temp['indName_recorded'].isnull().sum()))

Number of observations with no industry entry:  441
Number of observations whose industry has not been identified:  441


# Deal with the observations without industry name entry: Jaccard Similarity

In [27]:
mask = df_FDI['twIndName_official'].isnull()
print(df_FDI[['國別', 'indName_recorded', '核准日期','twIndName_official']][mask])

          國別 indName_recorded     核准日期 twIndName_official
44        中國              NaN  1040511                NaN
53        中國              NaN  1040507                NaN
84    英屬開曼群島              NaN   980629                NaN
85        中國              NaN  1040909                NaN
90        中國              NaN  1040909                NaN
...      ...              ...      ...                ...
9370      中國              NaN  1040126                NaN
9407      香港              NaN  1000210                NaN
9434      中國              NaN  1030925                NaN
9472      中國              NaN  1040330                NaN
9499      香港              NaN   980501                NaN

[441 rows x 4 columns]


In [120]:
import jieba

def chinese_tokenizer(text):
    return jieba.cut(text, cut_all=False)

def jaccard_similarity(str1, str2):
    words1 = chinese_tokenizer(str1)
    words2 = chinese_tokenizer(str2)
    set1 = set(words1)
    set2 = set(words2)
    return len(set1.intersection(set2)) / len(set1.union(set2))

print(jaccard_similarity("酒要一口一口喝", "要一口一口喝"))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\daeyu\AppData\Local\Temp\jieba.cache
Loading model cost 0.961 seconds.
Prefix dict has been built successfully.


0.5


In [ ]:
# 現在發現: 不確定"主要營業項目"是指總公司的主要營業項目，還是指海外投資計劃的營業項目。
# 例: 墨西哥-金融控股業-振樺電子股份有限公司-電子產品及週邊設備之買賣.一般進出口貿易業務
# 例: 英屬開曼群島-投資顧問業-振樺電子股份有限公司-投資業務

for i in df_FDI.index:
    if pd.isna(df_FDI['twIndCode_4d'][i]) == True:
        
        if pd.isna(df_FDI['業別'][i]) == True:
            # print(df_FDI['國內投資人'][i], end=";")
            # print(df_FDI['核准日期'][i])
            
            if pd.isna(df_FDI['國內投資人'][i])==False and pd.isna(df_FDI['核准日期'][i])==False and pd.isna(df_FDI['主要營業項目'][i])==False:
                print("The project of " + df_FDI['國內投資人'][i] + " on " + str(df_FDI['核准日期'][i]) + " does not record industry.")
                
                industry_candidate = ""
                industry_candidateIndex = 0
                jacSim_max = 0
                for j in df_FDI.index:
                    if pd.isna(df_FDI['twIndCode_4d'][j]) == False and pd.isna(df_FDI['主要營業項目'][j])==False:
                        jacSim = jaccard_similarity(df_FDI['主要營業項目'][i], df_FDI['主要營業項目'][j])
                        if jacSim > jacSim_max:
                            industry_candidate = df_FDI['業別'][j]
                            industry_candidateIndex = j
                            jacSim_max = jacSim
                print("This project, with operation " + df_FDI['主要營業項目'][i] + ", is identified as" + industry_candidate + " due to the similarity with " + df_FDI['國內投資人'][industry_candidateIndex] + "'s project on " + df_FDI['核准日期'][industry_candidateIndex] + "with main operation " + df_FDI['主要營業項目'][industry_candidateIndex]  + ", with jaccardSim = " + str(jacSim_max))
                print("------------------------------------------") 
        else:
            continue
    
    else:
        continue

# From Taiwan industry classification to ISIC industry classification.